# Try Applying to ClearWater
Building on the Example From: https://xarray-simlab.readthedocs.io/
And on the Example Notebook: https://github.com/EcohydrologyTeam/ClearWater-modules/blob/simlab/examples/dev_sandbox/xsimlab_examples.ipynb

# Imports
Using the environment that you set up in the example notebook. Run the following. 


In [47]:
import numpy as np

import matplotlib.pyplot as plt
import xsimlab as xs
import xarray as xr

import datetime

In [70]:
#create some datetime indices

hourly = xr.date_range("2025-01-01", "2025-01-02", freq="h")
minute_5 = xr.date_range("2025-01-01", "2025-01-02", freq="5min")

print(hourly.shape)
print(minute_5.shape)

(25,)
(289,)


In [ ]:


cordtime_ds = xr.Dataset(
    {
        "temp": xr.DataArray(
            np.array([minute_5, np.random.rand(289)], dtype=(np.datetime64, np.float64)),
            #coords={"index": "time"},
            dims=("time","temp"),
        ),
        "solar": xr.DataArray(
            data,
            #coords={"index_mod1": solar[0]},
            dims=("time_mod1","solar"),
            
        ),
    }
)




TypeError: Field elements must be 2- or 3-tuples, got '<class 'numpy.datetime64'>'

In [79]:
cordtime_ds


<xarray.Dataset> Size: 5kB
Dimensions:  (time: 2, temp: 289, time_mod1: 10, solar: 2)
Coordinates:
    temp     (time, temp) object 5kB 1735689600000000000 ... 0.2734204354213964
    solar    (time_mod1, solar) float64 160B -0.4054 -0.8525 ... 0.4363 0.7963
Dimensions without coordinates: time, time_mod1
Data variables:
    *empty*

In [46]:

cordtime_ds["temp"].indexes

Indexes:
    *empty*

In [ ]:
# we want parameters that are one xarray dataset
# without a shared time dimension

# solar radiation
solar = np.ndarray(
    ()
    ()
    ()
)

ds = xr.Dataset(
    {
        "time": xr.DataArray(np.arange(0, 10, 1), dims=("time")),
        "solar": xr.DataArray(solar, dims=("time", "solar")),
    }
)

ValueError: different number of dimensions on data and dims: 0 vs 2

We also need to import the clearwater_riverine module. 
This should have been in installed with conda develop, but may not have been if you only followed the example notebook.
Uncomment out the line below and run it.

In [ ]:
#note you will need to adjust the path to your repo location
#!conda develop "C:\Users\ptomasula\Repositories\ClearWater-riverine\src"

added C:\Users\ptomasula\Repositories\ClearWater-riverine\src
completed operation for: C:\Users\ptomasula\Repositories\ClearWater-riverine\src


In [ ]:
# I was also missing a dependency for riverine when I was working off the example environment
#!conda install h5py

In [5]:
import clearwater_riverine as cwr

# Wrap ClearWater-Riverine into xarray simlab 

In [268]:
@xs.process
class Riverine:
    riverine_path = xs.variable(
        dims=('path'),
        intent='in',
        description="Path to the configuration file for the riverine model",
    )
    #This should eventually be a model parameter and not specific to the process
    datetime_range = xs.variable(
        dims=('indicies'),
        intent='in',
        description="Datetime range for the simulation",
    )
    mesh = xs.variable(
        dims=('time', 'nface'),
        intent='out',
        description="Mesh for the riverine model",
    )

    def initialize(self):
        config_filepath = self.riverine_path[0]
        print(config_filepath)
        start_index, end_index = self.datetime_range
        print(start_index, end_index)

        self._transport = cwr.ClearwaterRiverine(
            config_filepath=config_filepath,
            datetime_range=(960, 1920)
        )

    @xs.runtime(args="step_delta")
    def run_step(self, dt):
        print("Running step")
        self._transport.update()

    def finalize_step(self):
        self.mesh = self._transport.mesh
        print(self.mesh)

# Build the model
In this case we only have a single process, which is our ClearWater Riverine transport model.

In [269]:
model = xs.Model({"transport": Riverine})
model

<xsimlab.Model (1 processes, 2 inputs)>
transport
    riverine_path      [in] ('path',) Path to the configuration fil...
    datetime_range     [in] ('indicies',) Datetime range for the si...

To run the model, we also need to specify an input dataset.

## Riverine Configuration
The ClearWater model requires on configuration file to initialize. 

In [270]:
# Our transport model needs a configuration pathway
config_path = r"./data_temp/sumwere_creek_coarse_p48/demo_config.yml"
path =xr.DataArray(
    [config_path],
    dims=('path'),
)

## Model Start and End

In [271]:
start_index = int(8*60*(60/30))  # start at 8:00 am on the first day of the simulation (30 second model)
end_index = start_index + int(8*60*(60/30))  # end 8 hours later (30 second model)start = 

In [174]:
# This is in the coupling example, but it is required for the tsm module
# not necessary for basic advection.
#mesh_path = r'./data_temp/sumwere_creek_coarse_p48/wetted_surface_area.zarr'
#wetted_sa = xr.open_zarr(mesh_path).compute()
#wetted_sa_subset = wetted_sa.isel(time=slice(start_index, end_index+1))
#mesh = xr.DataArray(
#    wetted_sa_subset['wetted_surface_area'].values,
#    dims=('time', 'nface')
#)

In [ ]:
#mesh

<xarray.DataArray (time: 961, nface: 444)> Size: 3MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(961, 444))
Dimensions without coordinates: time, nface

In [275]:
input_dataset = xs.create_setup(
    model=model,
    clocks={'step': np.arange(100)},
    input_vars={
        'transport__riverine_path': path, #path,
        'transport__datetime_range': ('indicies' , [start_index,end_index]),
        #'transport__mesh': (mesh.values),
    },
    #output_vars={
    #   'transport__mesh': 'step'
    #},
)

C:\Users\ptomasula\Repositories\xarray-simlab\xsimlab\xr_accessor.py:259: FutureWarning: variable 'step' with name matching its dimension will not be automatically converted into an `IndexVariable` object in the future.
  xr_var = as_variable(data, name=dim)


In [276]:
output_dataset = input_dataset.xsimlab.run(model=model)
output_dataset

./data_temp/sumwere_creek_coarse_p48/demo_config.yml
960 1920
'Cell Hydraulic Depth' not found in hdf file; skip reading it. 
'Cell Velocity - Velocity X' not found in hdf file; skip reading it. 
'Cell Velocity - Velocity Y' not found in hdf file; skip reading it. 
Cell velocities X and Y not found in hdf file; skip calculating velocity magnitude
Running step
<xarray.Dataset> Size: 28MB
Dimensions:                 (node: 549, time: 961, nface: 444, nmax_face: 8,
                             nedge: 915, 2: 2)
Coordinates:
    node_x                  (node) float64 4kB 5.004e+05 5.005e+05 ... 5.024e+05
    node_y                  (node) float64 4kB 2.381e+03 2.376e+03 ... 500.0
  * time                    (time) datetime64[ns] 8kB 2022-05-13T08:00:00 ......
    face_x                  (nface) float64 4kB 5.005e+05 ... 5.024e+05
    face_y                  (nface) float64 4kB 2.45e+03 2.45e+03 ... 564.4
Dimensions without coordinates: node, nface, nmax_face, nedge, 2
Data variables: (12/1

<xarray.Dataset> Size: 1kB
Dimensions:                    (step: 100, indicies: 2, path: 1)
Coordinates:
  * step                       (step) int64 800B 0 1 2 3 4 5 ... 95 96 97 98 99
Dimensions without coordinates: indicies, path
Data variables:
    transport__datetime_range  (indicies) int64 16B 960 1920
    transport__riverine_path   (path) <U52 208B './data_temp/sumwere_creek_co...

In [4]:

ds = xr.DataArray(np.random.rand(100, 2)*100, dims=('step', 'temperature'))

a0 = 6984.505294
a1 = -188.903931
a2 = 2.133357675
a3 = -1.288580973E-2
a4 = 4.393587233E-5
a5 = -8.023923082E-8
a6 = 6.136820929E-11

#this is an 5th order polynomial with as a constants as coefficients
ds2 = (
    + a1 * ds**0
    + 1*a2 * ds**1
    + 2*a3 * ds**2
    + 3*a4 * ds**3
    + 4*a5 * ds**4
    + 5*a6 * ds**5
)
print(ds[1:10,:])
print(ds2[1:10,:])

<xarray.DataArray (step: 9, temperature: 2)> Size: 144B
array([[16.6838787 , 56.5480937 ],
       [94.25863483, 79.87038691],
       [27.23839297, 51.15703296],
       [31.9695194 , 51.18661277],
       [13.51190342, 10.37998705],
       [97.56445576, 67.75443193],
       [44.44071443, 44.35389949],
       [ 5.06273148, 31.66375247],
       [24.38608036, 47.75087403]])
Dimensions without coordinates: step, temperature
<xarray.DataArray (step: 9, temperature: 2)> Size: 144B
array([[-159.89718525, -129.94675848],
       [-129.45849784, -127.82222   ],
       [-147.42381081, -131.65737248],
       [-143.05967808, -131.6464361 ],
       [-164.46877754, -169.39272429],
       [-130.0387119 , -127.99694516],
       [-134.62439042, -134.66936168],
       [-178.74697869, -143.32078348],
       [-150.40498258, -133.03871479]])
Dimensions without coordinates: step, temperature


In [15]:
ds = xr.DataArray(np.random.rand(100, 2)*100, dims=('step', 'temperature'))

vector = np.array((2,.5), dtype=np.float64)

print(ds[0:10,:])
result = (ds * vector*2)
print(result[0:10,:])

<xarray.DataArray (step: 10, temperature: 2)> Size: 160B
array([[12.68196634, 30.84120452],
       [ 0.69471472, 56.50463091],
       [ 7.63350829, 10.39982877],
       [28.70926736, 92.11570406],
       [17.95252805, 17.59870953],
       [58.03534572, 66.39496965],
       [84.34645866, 58.28810725],
       [64.28869412, 39.71019884],
       [49.80924611, 64.48494457],
       [63.68670334,  4.03766435]])
Dimensions without coordinates: step, temperature
<xarray.DataArray (step: 10, temperature: 2)> Size: 160B
array([[ 50.72786537,  30.84120452],
       [  2.77885886,  56.50463091],
       [ 30.53403317,  10.39982877],
       [114.83706946,  92.11570406],
       [ 71.81011219,  17.59870953],
       [232.14138287,  66.39496965],
       [337.38583463,  58.28810725],
       [257.15477646,  39.71019884],
       [199.23698445,  64.48494457],
       [254.74681336,   4.03766435]])
Dimensions without coordinates: step, temperature
